# Web Scraping

**Web Scraping** is the process of crawling across a webpage and collecting the text content. Some popular services, such as SkyScanner, collect data from many sources and display it all together for their users to compare. 

Web scraping is often considered an option if there is no API, but beware that it lies in a grey area in terms of ethics and legality. You should not scrape data that isn't publicly available, and many websites have implemented protection measures to prevent web scrapers from taking their data. But in many cases, it is a perfectly acceptable data collection method, especially as a learning exercise. 

If you want to check a website's policy on scraping, most websites have a file called [`robots.txt`](https://www.promptcloud.com/blog/how-to-read-and-respect-robots-file/), which will be the base URL + "/robots.txt". This will contain rules to communicate to bots crawling the site what they should and shouldn't do. 

## HTML

The main content of a webpage is displayed in **HTML** (HyperText Markup Language) files. Web browsers format the HTML elements, but additional styling and functionality can be added with **CSS** (Cascading Style Sheet) and **JavaScript**. The code for CSS and JavaScript are commonly written in separate files and imported into the HTML to save space, but not always! 

We can use the inspector tool in a web browser to look at the HTML for any webpage. You could also consider using a tool like [Live DOM Viewer](https://software.hixie.ch/utilities/js/live-dom-viewer/). **DOM** = Document Object Model, which is the tree of elements that make up an HTML document.

If you want to create your own `.html` file and experiment:
  - Create a file called `index.html`, and boilerplate an HTML document with `!`
  - `<head>` tag holds metadata, `<body>` tag holds elements that render on the page
  <!-- - Add `<style>` tag to demonstrate CSS (and also how a separate file could be linked)
  - Add `<script>` tag to demonstrate JS (and also how a separate file could be linked) -->
  - Explore some basic tags and common attributes (`<h1>`, `<p>`, `<a>`, `<img>`)
  - Explore HTML tag nesting (`<div>`, `<ul>`)

Now that we know what we're looking for, we can look at some Python libraries to help us achieve it! Choosing which library will depend on the website you are looking at, and how the page content is generated and displayed. 

There are two main types of webpages you'll find online: **static** and **dynamic**. Static refers to HTML pages that are "finished" on arrival, all the content is already available when the document is received. Dynamic pages use JavaScript to build the HTML _after_ the browser receives the document, meaning there will be a short (often barely even perceptible to humans) time in which the page doesn't have the content. 

There is also the question of accessing the exact data you are after - will you need to interact with elements on the webpage to navigate, filter, search, etc? 

## Beautiful Soup

[**Beautiful Soup**](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is our recommended library for static sites. It is a lightweight package designed for parsing HTML and XML files. You should add `beautifulsoup4` (bs4) to your Conda environment for this project. You will also need to add [`requests`](https://pypi.org/project/requests/) to make the HTTP requests, and (optional) [`lxml`](https://lxml.de/) to help process the HTML/XML. 

In [4]:
import requests
import bs4

response = requests.get("https://example.com/")
# soup = bs4.BeautifulSoup(response.text, 'lxml') # with lxml package
doc = bs4.BeautifulSoup(response.text, 'html.parser') 
doc

<!DOCTYPE html>

<html>
<head>
<title>Example Domain</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>
</head>
<body>
<div>
<h1>Example Domain</h1>
<p>This domain is for use in illustrative example

Once we have some "soupified" HTML, we can target specific elements inside and collect the data content they are holding. There are actually many maaaany ways we can do this (refer to documentation), and we should specify if we want a single or multiple results: 

In [ ]:
# target first <p> element inside the body:
doc.body.p

# target first <p> element in the document:
doc.find('p')
doc.select_one('p')

# # target all <p> elements in the document:
doc.find_all('p')
doc.select('p')

The `select` and `select_one` methods will accept [CSS Selectors](https://www.freecodecamp.org/news/css-selectors-cheat-sheet/), while `find` and `find_all` will take [multiple arguments](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find_all), including the tag name and attributes (if you ever want to practise CSS selectors, [here](https://flukeout.github.io/) is a great resource!)

In [ ]:
# these both target <p class="red">this is a paragraph</p>
doc.find_all("p", attrs={"class": "red"})
doc.select("p.red")

Once you have selected an element, you can access the [attributes](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#bs4.Tag.attrs) and [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#get-text), and you can also [navigate](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#navigating-the-tree) to its surrounding elements (children, parents, siblings) through the DOM tree. 

The website [books.toscrape.com](https://books.toscrape.com/) is a dummy website build for the purpose of practising web scraping. We can build a small dataset by taking some of the content. I want to make a dataframe from the books, collecting their title, rating, price, and the url for the cover image. 

In [5]:
response = requests.get("https://books.toscrape.com/")
soup = bs4.BeautifulSoup(response.text, 'html.parser')

By inspecting the HTML, I can see that the cards holding the data for each book are `<li>` (list) elements nested inside an `<ol>` (ordered list) element with a class of "row", and that the elements holding the content I want are grouped into an `<article>` element with a class of "product_pod". It is important to make sure I am not going to end up with any other rogue elements in my list of books, as this could break my code. The more specific the combination, the less likely the chance of overlapping selectors. 

In [6]:
# break the selector down to show nesting
# book_cards = soup.select("article.product_pod")
book_cards = soup.find_all("article", attrs={"class": "product_pod"})
book_cards

[<article class="product_pod">
 <div class="image_container">
 <a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
 </div>
 <p class="star-rating Three">
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 <i class="icon-star"></i>
 </p>
 <h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
 <div class="product_price">
 <p class="price_color">Â£51.77</p>
 <p class="instock availability">
 <i class="icon-ok"></i>
     
         In stock
     
 </p>
 <form>
 <button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
 </form>
 </div>
 </article>,
 <article class="product_pod">
 <div class="image_container">
 <a href="catalogue/tipping-the-velvet_999/index.html"><img alt="Tipping the Velvet" class="th

This returns a list of the article elements, which I can loop over and extract the data I want. To get the title, I'll need to look at the `<a>` inside the `<h3>` (there is more than one `<a>` element in the card, so I have to be specific when selecting). You'll notice though that the text cuts off the end of the title if it's too long, so we want to take the value from the `title` attribute, rather than the text.

In [10]:
titles = []

for book in book_cards:
  # title = book.select_one('h3 > a')['title']
  # titles.append(title)
  title = book.find("h3").findChild("a")["title"]
  titles.append(title)


titles

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind',
 'The Requiem Red',
 'The Dirty Little Secrets of Getting Your Dream Job',
 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull',
 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics',
 'The Black Maria',
 'Starving Hearts (Triangular Trade Trilogy, #1)',
 "Shakespeare's Sonnets",
 'Set Me Free',
 "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)",
 'Rip it Up and Start Again',
 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991',
 'Olio',
 'Mesaerion: The Best Science Fiction Stories 1800-1849',
 'Libertarianism for Beginners',
 "It's Only the Himalayas"]

To find the the rating, I need to understand their system. It seems they are using a class of "One" to "Five" to determine how the stars display, so the second class on the `<p>` element with a first class of "star-rating" is what tells me the rating. The class attribute will be a list of all classes applied to the element, and since I only want the second, I will use the index. At this point, I could also write a simple function to convert the string to an Int for my data:

In [14]:
ratings = []

for book in book_cards:
  # rating = book.select_one('.star-rating')['class'][1]
  rating = book.find("p", attrs={"class": "star-rating"})["class"][1]
  ratings.append(rating)
  
ratings

['Three',
 'One',
 'One',
 'Four',
 'Five',
 'One',
 'Four',
 'Three',
 'Four',
 'One',
 'Two',
 'Four',
 'Five',
 'Five',
 'Five',
 'Three',
 'One',
 'One',
 'Two',
 'Two']

In [15]:
def convert_rating(string):
  return ["One", "Two", "Three", "Four", "Five"].index(string) + 1

ratings_int = [convert_rating(string) for string in ratings]
ratings_int

[3, 1, 1, 4, 5, 1, 4, 3, 4, 1, 2, 4, 5, 5, 5, 3, 1, 1, 2, 2]

To access the price, I can target the `<p>` element with a class of "price_color", and extract the text. At this point I could also convert this to a float (first remove the symbols). 

In [24]:
prices = []

for book in book_cards:
  # price = book.find("p", attrs={"price_color"}).get_text()
  # formatted_price = float(price.replace('Â£', ''))
  # prices.append(formatted_price)
  price_element = book.find("p", attrs={"class", "price_color"})
  if price_element:
    price = price_element.get_text()
    prices.append(price.replace("Â£", ""))
  
prices

['51.77',
 '53.74',
 '50.10',
 '47.82',
 '54.23',
 '22.65',
 '33.34',
 '17.93',
 '22.60',
 '52.15',
 '13.99',
 '20.66',
 '17.46',
 '52.29',
 '35.02',
 '57.25',
 '23.88',
 '37.59',
 '51.33',
 '45.17']

Finally, to access the cover image URL, I will need to target the `<img>` tag and extract it from the `src` attribute. You'll notice though that it isn't a complete URL. This is because the images are hosted on the same domain, so we can create a full URL that can be linked from anywhere by adding the website's base URL to the string we get from the image source (always double check your `/` symbols when concatenating URL strings):

In [26]:
covers = []

base_url = "https://books.toscrape.com/"

for book in book_cards:
  cover_extension = book.find("img")["src"]
  covers.append(base_url + cover_extension)
  
covers

['https://books.toscrape.com/media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg',
 'https://books.toscrape.com/media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg',
 'https://books.toscrape.com/media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg',
 'https://books.toscrape.com/media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg',
 'https://books.toscrape.com/media/cache/be/a5/bea5697f2534a2f86a3ef27b5a8c12a6.jpg',
 'https://books.toscrape.com/media/cache/68/33/68339b4c9bc034267e1da611ab3b34f8.jpg',
 'https://books.toscrape.com/media/cache/92/27/92274a95b7c251fea59a2b8a78275ab4.jpg',
 'https://books.toscrape.com/media/cache/3d/54/3d54940e57e662c4dd1f3ff00c78cc64.jpg',
 'https://books.toscrape.com/media/cache/66/88/66883b91f6804b2323c8369331cb7dd1.jpg',
 'https://books.toscrape.com/media/cache/58/46/5846057e28022268153beff6d352b06c.jpg',
 'https://books.toscrape.com/media/cache/be/f4/bef44da28c98f905a3ebec0b87be8530.jpg',
 'https://books.toscrape.com/media/cache/10/48/1048f63

Once I have my lists, I can create a DataFrame and [CSV](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)

In [27]:
import pandas as pd

books_df = pd.DataFrame({
  'title': titles,
  'price': prices,
  'rating': ratings,
  'cover': covers
})

books_df.to_csv("out.csv", index=False)

In [28]:
books_df.head()

,title,price,rating,cover
0,A Light in the Attic,51.77,Three,https://books.toscrape.com/media/cache/2c/da/2...
1,Tipping the Velvet,53.74,One,https://books.toscrape.com/media/cache/26/0c/2...
2,Soumission,50.10,One,https://books.toscrape.com/media/cache/3e/ef/3...
3,Sharp Objects,47.82,Four,https://books.toscrape.com/media/cache/32/51/3...
4,Sapiens: A Brief History of Humankind,54.23,Five,https://books.toscrape.com/media/cache/be/a5/b...


This has only collected data from the _first_ page though. If we look at the website, we can navigate to page 2 with the click of a button. Notice the URL indicates which page we are on - this is a great opportunity to loop and gather not just the first 50, but all 1000 books listed on the website! This site is nice enough to indicate how many pages we have to iterate over, but many websites won't give you this. It is also likely to change as stock levels rise and drop! So we have to come up with a dynamic solution. 

If the total page number is displayed on the website, you can scrape that value and `for` loop over a [range](https://www.w3schools.com/python/ref_func_range.asp). If it isn't displayed, then we're going to need to use a while loop until there are no more pages. If we open [page 51](https://books.toscrape.com/catalogue/page-51.html), we get an error 404 page. This error status code will be accessible in the response object returned by `requests.get()`, which we can set to the condition of the loop.

We can format the URL string using Python's [string_format](https://www.w3schools.com/python/ref_string_format.asp):

In [ ]:
page_number = 40
while True:
  url = f"{base_url}catalogue/page-{page_number}.html"
  response = requests.get(url)
  print(page_number, response)
  page_number += 1
  if response.status_code != 200:
    break
  

40 <Response [200]>
41 <Response [200]>
42 <Response [200]>
43 <Response [200]>
44 <Response [200]>
45 <Response [200]>
46 <Response [200]>
47 <Response [200]>
48 <Response [200]>
49 <Response [200]>
50 <Response [200]>
51 <Response [404]>


Be aware that Beautiful Soup is only going to work for static webpages. If we try to get the HTML for a dynamic page (take the LMS for example, which is a React App, meaning the page content is generated in the browser when you open it), you'll get a mostly empty page, and some JavaScript nonsense. 

In [33]:
response = requests.get("https://lms.codeacademyberlin.com/")
soup = bs4.BeautifulSoup(response.text, 'lxml')
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><link href="/favicon.ico" rel="icon"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="#000000" name="theme-color"/><meta content="LMS | Code Academy Berlin" name="description"/><link href="/logo192.png" rel="apple-touch-icon"/><link href="/manifest.json" rel="manifest"/><title>LMS | Code Academy Berlin</title><link href="/static/css/2.395b84fd.chunk.css" rel="stylesheet"/><link href="/static/css/main.89931962.chunk.css" rel="stylesheet"/></head><body><style>.loader{display:block;position:absolute;top:50%;left:50%;transform:translate(-50%,-50%);width:200px;height:200px;animation:blink 1.5s linear infinite alternate}@keyframes blink{0%{opacity:0}50%{opacity:.5}100%{opacity:1}}</style><noscript>You need to enable JavaScript to run this app.</noscript><div class="loader"><svg id="svg2" viewbox="0 0 525.36 476.04" xmlns="http://www.w3.org/2000/svg"><title>PenroseLogo</title><g id="layer1"><path d

## Your Task! 

You will use BeautifulSoup to scrape data from [U.Today](https://u.today/) financial news. Use the search feature to find results specific to your chosen cryptocurrency. You can also add additional filters, notice how they are added to the URL! The purpose of this exercise is to collect data, don't worry when the data you're collecting is not the most relevant.